# Demo: Multi-Step Agent with Tools

**NLP Final Lecture - Live Demo**

This notebook demonstrates a simple ReAct-style agent that can:
1. Think about what to do
2. Call tools (calculator, search simulator)
3. Observe results
4. Continue until task is complete

In [ ]:
import os
import json
from openai import OpenAI

# Set your API key
# os.environ["OPENAI_API_KEY"] = "your-key-here"

client = OpenAI()

## Define Tools

We'll create simple tools that the agent can use.

In [ ]:
# Tool definitions for OpenAI function calling
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Perform mathematical calculations. Use for any math operations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Mathematical expression to evaluate, e.g., '2 + 2' or '15 * 0.20'"
                    }
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_info",
            "description": "Search for factual information. Use when you need to look up data.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_date",
            "description": "Get the current date and time.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    }
]

print(f"Defined {len(tools)} tools for the agent")

In [ ]:
# Tool implementations
from datetime import datetime

def calculator(expression: str) -> str:
    """Safely evaluate a mathematical expression."""
    try:
        # Only allow safe math operations
        allowed = set('0123456789+-*/.() ')
        if not all(c in allowed for c in expression):
            return "Error: Invalid characters in expression"
        result = eval(expression)
        return f"{result}"
    except Exception as e:
        return f"Error: {str(e)}"

def search_info(query: str) -> str:
    """Simulated search - returns mock data for demo."""
    # Simulated knowledge base
    knowledge = {
        "apple stock price": "Apple (AAPL) current price: $178.50",
        "apple market cap": "Apple market cap: $2.75 trillion (as of 2025)",
        "deepseek r1": "DeepSeek-R1 is an open-source reasoning model released in January 2025",
        "population of france": "France population: approximately 68 million",
        "euro to usd": "Exchange rate: 1 EUR = 1.08 USD",
        "default": "I found some information but it may not be exactly what you're looking for."
    }
    
    query_lower = query.lower()
    for key, value in knowledge.items():
        if key in query_lower:
            return value
    return knowledge["default"]

def get_current_date() -> str:
    """Return current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Map function names to implementations
tool_functions = {
    "calculator": calculator,
    "search_info": search_info,
    "get_current_date": get_current_date
}

# Test tools
print("Testing tools:")
print(f"Calculator: 15 * 1.08 = {calculator('15 * 1.08')}")
print(f"Search: {search_info('apple market cap')}")
print(f"Date: {get_current_date()}")

## The Agent Loop

In [ ]:
def run_agent(user_query: str, max_steps: int = 5) -> str:
    """Run a ReAct-style agent loop."""
    
    messages = [
        {
            "role": "system",
            "content": """You are a helpful assistant with access to tools. 
Think step by step about how to answer the user's question.
Use tools when you need to look up information or perform calculations.
When you have enough information, provide a clear final answer."""
        },
        {"role": "user", "content": user_query}
    ]
    
    print(f"\n{'='*60}")
    print(f"USER: {user_query}")
    print(f"{'='*60}\n")
    
    for step in range(max_steps):
        print(f"--- Step {step + 1} ---")
        
        # Call the model
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=0
        )
        
        assistant_message = response.choices[0].message
        
        # Check if model wants to use tools
        if assistant_message.tool_calls:
            # Add assistant message to history
            messages.append(assistant_message)
            
            # Process each tool call
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"THOUGHT: Need to call {function_name}")
                print(f"ACTION: {function_name}({function_args})")
                
                # Execute the tool
                if function_name in tool_functions:
                    result = tool_functions[function_name](**function_args)
                else:
                    result = f"Unknown tool: {function_name}"
                
                print(f"OBSERVATION: {result}\n")
                
                # Add tool result to messages
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": result
                })
        else:
            # Model gave a final answer
            final_answer = assistant_message.content
            print(f"FINAL ANSWER: {final_answer}")
            return final_answer
    
    return "Max steps reached without final answer"

## Demo: Multi-Step Task

In [ ]:
# Task requiring multiple tools
query = "What is 15% of Apple's current market cap?"

result = run_agent(query)

In [ ]:
# Another multi-step task
query = "I have 500 euros. How many US dollars is that, and what's 20% of that amount?"

result = run_agent(query)

In [ ]:
# Simple task (may not need tools)
query = "What is the capital of Japan?"

result = run_agent(query)

## Visualizing the Agent Trace

In [ ]:
def run_agent_with_trace(user_query: str, max_steps: int = 5):
    """Run agent and return detailed trace."""
    
    trace = {
        "query": user_query,
        "steps": [],
        "final_answer": None
    }
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant with tools. Think step by step."},
        {"role": "user", "content": user_query}
    ]
    
    for step in range(max_steps):
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=0
        )
        
        assistant_message = response.choices[0].message
        
        if assistant_message.tool_calls:
            messages.append(assistant_message)
            
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                result = tool_functions[function_name](**function_args)
                
                trace["steps"].append({
                    "type": "tool_call",
                    "tool": function_name,
                    "args": function_args,
                    "result": result
                })
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": result
                })
        else:
            trace["final_answer"] = assistant_message.content
            break
    
    return trace

# Get trace
trace = run_agent_with_trace("What is 25% of the population of France?")

# Display trace nicely
print("\nAgent Execution Trace:")
print(f"Query: {trace['query']}\n")
for i, step in enumerate(trace['steps']):
    print(f"Step {i+1}: {step['tool']}({step['args']})")
    print(f"  Result: {step['result']}")
print(f"\nFinal Answer: {trace['final_answer']}")

## Key Takeaways

1. **Agents = LLMs + Tools + Loop**: The model decides when/how to use tools
2. **ReAct pattern**: Think -> Act -> Observe -> Repeat
3. **Tool use is learned**: Models trained on function calling know the format
4. **Challenges**: Reliability, cost, infinite loops, wrong tool selection

## Connection to RAG

RAG is essentially an agent with a single tool: retrieval. Agents generalize this to multiple tools!